**Download libraries**

In [ ]:
!pip install torch
!pip install torchvision
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**RESNET**

In [ ]:
import torch
from torch import nn, optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import numpy as np

data_dir = '/content/drive/MyDrive/Dataset_aug'

# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace the last layer to match the number of classes
model.fc = nn.Linear(model.fc.in_features, 5)

# Set up data transformations
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
image_datasets = ImageFolder(data_dir, transform=data_transforms)

# Determine the sizes of the training and validation sets
train_size = int(0.8 * len(image_datasets))
val_size = len(image_datasets) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(image_datasets, [train_size, val_size])

# Create DataLoaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs} loss: {loss.item()}')

# Save the trained model
torch.save(model, 'resnet.pth')

# After training, evaluate on the validation set
model.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())

# Generate classification report
report = classification_report(true_labels, pred_labels, output_dict=True)

# Save classification report to CSV
df = pd.DataFrame(report).transpose()
df.to_csv('classification_report_resnet.csv')

# Generate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Visualize confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix_resnet.png')



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 201MB/s]


Epoch 1/25 loss: 1.3965370655059814
Epoch 2/25 loss: 1.012216567993164
Epoch 3/25 loss: 0.764329195022583
Epoch 4/25 loss: 1.138136863708496
Epoch 5/25 loss: 0.8742983341217041
Epoch 6/25 loss: 0.74367356300354
Epoch 7/25 loss: 0.8146423101425171
Epoch 8/25 loss: 0.7722604274749756
Epoch 9/25 loss: 0.7451421022415161
Epoch 10/25 loss: 0.650442361831665
Epoch 11/25 loss: 1.028049111366272
Epoch 12/25 loss: 1.0574769973754883
Epoch 13/25 loss: 1.239819049835205
Epoch 14/25 loss: 0.8107497096061707
Epoch 15/25 loss: 0.8404054045677185
Epoch 16/25 loss: 0.625316321849823
Epoch 17/25 loss: 0.5812388062477112
Epoch 18/25 loss: 1.0164140462875366
Epoch 19/25 loss: 0.808189868927002
Epoch 20/25 loss: 0.9119093418121338
Epoch 21/25 loss: 0.8127294778823853
Epoch 22/25 loss: 0.9642001390457153
Epoch 23/25 loss: 0.796804666519165
Epoch 24/25 loss: 0.7294995784759521
Epoch 25/25 loss: 0.5784299969673157


In [ ]:
# First, store the class names in a variable
class_names = image_datasets.classes

# Then, print each class index with its corresponding name
for i, class_name in enumerate(class_names):
    print(f'Index {i} corresponds to class: {class_name}')


**NASNetLarge**

In [ ]:
import torch
from torch import nn, optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import numpy as np


data_dir = '/content/drive/MyDrive/Dataset_aug'

# Load a pre-trained NASNet model
model = models.mnasnet1_0(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

model.classifier[1] = nn.Linear(model.classifier[1].in_features, 5)


# Set up data transformations
data_transforms = transforms.Compose([
    transforms.Resize(331),  # NASNetLarge requires the input size to be 331x331
    transforms.CenterCrop(331),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
image_datasets = ImageFolder(data_dir, transform=data_transforms)

# Determine the sizes of the training and validation sets
train_size = int(0.8 * len(image_datasets))
val_size = len(image_datasets) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(image_datasets, [train_size, val_size])

# Create DataLoaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs} loss: {loss.item()}')

# Save the trained model
torch.save(model, 'nasnet.pth')

# After training, evaluate on the validation set
model.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())

class_names = image_datasets.classes
true_labels_names = [class_names[label] for label in true_labels]
pred_labels_names = [class_names[label] for label in pred_labels]

# Generate classification report
report = classification_report(true_labels_names, pred_labels_names, output_dict=True)

# Save classification report to CSV
df = pd.DataFrame(report).transpose()
df.to_csv('classification_report_nasnet.csv')

# Generate confusion matrix
cm = confusion_matrix(true_labels_names, pred_labels_names)

# Visualize confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix_nasnet.png')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MNASNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=MNASNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mnasnet1.0_top1_73.512-f206786ef8.pth" to /root/.cache/torch/hub/checkpoints/mnasnet1.0_top1_73.512-f206786ef8.pth
100%|██████████| 16.9M/16.9M [00:00<00:00, 80.3MB/s]


In [ ]:
# First, store the class names in a variable
class_names = image_datasets.classes

# Then, print each class index with its corresponding name
for i, class_name in enumerate(class_names):
    print(f'Index {i} corresponds to class: {class_name}')

**SENET**

In [ ]:
!pip install pretrainedmodels


In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import numpy as np
import pretrainedmodels


import torch.hub
model = torch.hub.load(
    'moskomule/senet.pytorch',
    'se_resnet50',
    pretrained=True,)

data_dir = '/content/drive/MyDrive/Dataset_aug'

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 5)



# Set up data transformations
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
image_datasets = ImageFolder(data_dir, transform=data_transforms)

# Determine the sizes of the training and validation sets
train_size = int(0.8 * len(image_datasets))
val_size = len(image_datasets) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(image_datasets, [train_size, val_size])

# Create DataLoaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs} loss: {loss.item()}')

# Save the trained model
torch.save(model, 'senet.pth')

# After training, evaluate on the validation set
model.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())

# Generate classification report
report = classification_report(true_labels, pred_labels, output_dict=True)

# Save classification report to CSV
df = pd.DataFrame(report).transpose()
df.to_csv('classification_report_senet.csv')

# Generate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Visualize confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix SENet')
plt.savefig('confusion_matrix_senet.png')


In [ ]:
# First, store the class names in a variable
class_names = image_datasets.classes

# Then, print each class index with its corresponding name
for i, class_name in enumerate(class_names):
    print(f'Index {i} corresponds to class: {class_name}')

In [ ]:
!pip install efficientnet_pytorch


**Efficient NET**

In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import numpy as np
from efficientnet_pytorch import EfficientNet


data_dir = '/content/drive/MyDrive/Dataset_aug'

# Load a pre-trained EfficientNet model
model_name = 'efficientnet-b0'  # You can increase the number for more complex models
model = EfficientNet.from_pretrained(model_name)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

model._fc = nn.Linear(model._fc.in_features, 5)

# Set up data transformations
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
image_datasets = ImageFolder(data_dir, transform=data_transforms)

# Determine the sizes of the training and validation sets
train_size = int(0.8 * len(image_datasets))
val_size = len(image_datasets) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(image_datasets, [train_size, val_size])

# Create DataLoaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model._fc.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs} loss: {loss.item()}')

# Save the trained model
torch.save(model, 'efficientnet.pth')

# After training, evaluate on the validation set
model.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())

# Generate classification report
report = classification_report(true_labels, pred_labels, output_dict=True)

# Save classification report to CSV
df = pd.DataFrame(report).transpose()
df.to_csv('classification_report_efficientnet.csv')

# Generate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Visualize confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix EfficientNet')
plt.savefig('confusion_matrix_efficientnet.png')


In [ ]:
# First, store the class names in a variable
class_names = image_datasets.classes

# Then, print each class index with its corresponding name
for i, class_name in enumerate(class_names):
    print(f'Index {i} corresponds to class: {class_name}')

**MobileNet**

In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import numpy as np

data_dir = '/content/drive/MyDrive/Dataset_aug'

# Load a pre-trained MobileNet model
model = models.mobilenet_v2(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False


model.classifier[1] = nn.Linear(model.classifier[1].in_features, 5)

# Set up data transformations
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
image_datasets = ImageFolder(data_dir, transform=data_transforms)

# Determine the sizes of the training and validation sets
train_size = int(0.8 * len(image_datasets))
val_size = len(image_datasets) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(image_datasets, [train_size, val_size])

# Create DataLoaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[1].parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs} loss: {loss.item()}')

# Save the trained model
torch.save(model, 'mobilenet.pth')

# After training, evaluate on the validation set
model.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())

# Generate classification report
report = classification_report(true_labels, pred_labels, output_dict=True)

# Save classification report to CSV
df = pd.DataFrame(report).transpose()
df.to_csv('classification_report_mobilenet.csv')

# Generate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Visualize confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix MobileNet')
plt.savefig('confusion_matrix_mobilenet.png')


In [ ]:
# First, store the class names in a variable
class_names = image_datasets.classes

# Then, print each class index with its corresponding name
for i, class_name in enumerate(class_names):
    print(f'Index {i} corresponds to class: {class_name}')

**Inception V3**

In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import numpy as np

# Assume your images are in a directory called 'dataset' with subdirectories for each emotion
data_dir = '/content/drive/MyDrive/Dataset_aug'

# Load a pre-trained Inception v3 model
model = models.inception_v3(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace the last layers to match the number of classes
model.fc = nn.Linear(model.fc.in_features, 5)
model.AuxLogits.fc = nn.Linear(model.AuxLogits.fc.in_features, 5)

# Set up data transformations
data_transforms = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
image_datasets = ImageFolder(data_dir, transform=data_transforms)

# Determine the sizes of the training and validation sets
train_size = int(0.8 * len(image_datasets))
val_size = len(image_datasets) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(image_datasets, [train_size, val_size])

# Create DataLoaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam([
    {'params': model.fc.parameters()},
    {'params': model.AuxLogits.fc.parameters()}
], lr=0.001)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Adjust model for training
        model.train()
        outputs, aux_outputs = model(inputs)
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_outputs, labels)
        loss = loss1 + 0.4*loss2
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs} loss: {loss.item()}')

# Save the trained model
torch.save(model, 'inception_v3.pth')

# After training, evaluate on the validation set
model.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())

# Generate classification report
report = classification_report(true_labels, pred_labels, output_dict=True)

# Save classification report to CSV
df = pd.DataFrame(report).transpose()
df.to_csv('classification_report_inception.csv')

# Generate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Visualize confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix Inception v3')
plt.savefig('confusion_matrix_inception.png')


In [ ]:
# First, store the class names in a variable
class_names = image_datasets.classes

# Then, print each class index with its corresponding name
for i, class_name in enumerate(class_names):
    print(f'Index {i} corresponds to class: {class_name}')

**DenseNET**

In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import numpy as np

data_dir = '/content/drive/MyDrive/Dataset_aug'

# Load a pre-trained DenseNet model
model = models.densenet121(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False


model.classifier = nn.Linear(model.classifier.in_features, 5)

# Set up data transformations
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
image_datasets = ImageFolder(data_dir, transform=data_transforms)

# Determine the sizes of the training and validation sets
train_size = int(0.8 * len(image_datasets))
val_size = len(image_datasets) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(image_datasets, [train_size, val_size])

# Create DataLoaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs} loss: {loss.item()}')

# Save the trained model
torch.save(model, 'densenet.pth')

# After training, evaluate on the validation set
model.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())

# Generate classification report
report = classification_report(true_labels, pred_labels, output_dict=True)

# Save classification report to CSV
df = pd.DataFrame(report).transpose()
df.to_csv('classification_report_densenet.csv')

# Generate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Visualize confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix DenseNet')
plt.savefig('confusion_matrix_densenet.png')


In [ ]:
# First, store the class names in a variable
class_names = image_datasets.classes

# Then, print each class index with its corresponding name
for i, class_name in enumerate(class_names):
    print(f'Index {i} corresponds to class: {class_name}')

**ResNetX**

In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns


data_dir = '/content/drive/MyDrive/Dataset_aug'

# Load a pre-trained ResNeXt model
model = models.resnext50_32x4d(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False


num_classes = 5
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Set up data transformations
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
image_datasets = ImageFolder(data_dir, transform=data_transforms)

# Determine the sizes of the training and validation sets
train_size = int(0.8 * len(image_datasets))
val_size = len(image_datasets) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(image_datasets, [train_size, val_size])

# Create DataLoaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs} loss: {loss.item()}')

# Save the trained model
torch.save(model, 'resnext.pth')

# After training, evaluate on the validation set
model.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())

# Generate classification report
report = classification_report(true_labels, pred_labels, output_dict=True)

# Save classification report to CSV
df = pd.DataFrame(report).transpose()
df.to_csv('classification_report_resnext.csv')

# Generate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Visualize confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix ResNeXt')
plt.savefig('confusion_matrix_resnext.png')


In [ ]:
# First, store the class names in a variable
class_names = image_datasets.classes

# Then, print each class index with its corresponding name
for i, class_name in enumerate(class_names):
    print(f'Index {i} corresponds to class: {class_name}')

**ShuffleNET**


In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns

data_dir = '/content/drive/MyDrive/Dataset_aug'

# Load a pre-trained ShuffleNet model
model = models.shufflenet_v2_x1_0(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

num_classes = 5
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Set up data transformations
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
image_datasets = ImageFolder(data_dir, transform=data_transforms)

# Determine the sizes of the training and validation sets
train_size = int(0.8 * len(image_datasets))
val_size = len(image_datasets) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(image_datasets, [train_size, val_size])

# Create DataLoaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs} loss: {loss.item()}')

# Save the trained model
torch.save(model, 'shufflenet.pth')

# After training, evaluate on the validation set
model.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())

# Generate classification report
report = classification_report(true_labels, pred_labels, output_dict=True)

# Save classification report to CSV
df = pd.DataFrame(report).transpose()
df.to_csv('classification_report_shufflenet.csv')

# Generate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Visualize confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix ShuffleNet')
plt.savefig('confusion_matrix_shufflenet.png')


In [ ]:
# First, store the class names in a variable
class_names = image_datasets.classes

# Then, print each class index with its corresponding name
for i, class_name in enumerate(class_names):
    print(f'Index {i} corresponds to class: {class_name}')

**Wide ResNet**

In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns


data_dir = '/content/drive/MyDrive/Dataset_aug'

# Load a pre-trained Wide ResNet model
model = models.wide_resnet50_2(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

num_classes = 5
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Set up data transformations
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your dataset
image_datasets = ImageFolder(data_dir, transform=data_transforms)

# Determine the sizes of the training and validation sets
train_size = int(0.8 * len(image_datasets))
val_size = len(image_datasets) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(image_datasets, [train_size, val_size])

# Create DataLoaders for the training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs} loss: {loss.item()}')

# Save the trained model
torch.save(model, 'wide_resnet.pth')

# After training, evaluate on the validation set
model.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(preds.cpu().numpy())

# Generate classification report
report = classification_report(true_labels, pred_labels, output_dict=True)

# Save classification report to CSV
df = pd.DataFrame(report).transpose()
df.to_csv('classification_report_wide_resnet.csv')

# Generate confusion matrix
cm = confusion_matrix(true_labels, pred_labels)

# Visualize confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix Wide ResNet')
plt.savefig('confusion_matrix_wide_resnet.png')


In [ ]:
# First, store the class names in a variable
class_names = image_datasets.classes

# Then, print each class index with its corresponding name
for i, class_name in enumerate(class_names):
    print(f'Index {i} corresponds to class: {class_name}')